# Progressive Party Problem

**问题描述：** 你是一个游艇集结活动的组织者。有42艘游艇及其船员。此次游艇集结活动中一项重要的晚间活动，是组织一场派对，让每位船员都能尽可能多地与其他船员的成员进行交流。部分船只被选作举办派对的"主办船 (host)"，其余船只则为"访客船 (guest)"。主办船的船员在整个派对期间留守自家船只（不去其他船），而访客船的船员则轮流拜访各主办船。为了最大化交流机会，访客船船员在每个主办船仅停留30分钟，之后便前往另一艘主办船。整个派对持续3小时。

每个船员团队有既定的人数，且他们始终集体行动。同时，每艘船都有规定的容纳容量。当前的问题在于：需为所有六个时间段（即3小时/0.5小时）将所有的访客船员团队分配到各主办船上，并力求使用最少数量的主办船来完成整个安排。


 This problem was originally stated by Peter Hubbard (Southampton University), organizer of the yacht rally of 42 boats and their crews. A important evening event in the yacht rally is to organize a party where each crew socializes with as many other crews as possible. Some of the boats are selected to serve as “host boats” where the party takes place, the other boats are the “guest boats”. The crews of the host boats stay on their boats for the whole party, while the crews of the guest boats visits the host boats. In order to socialize as much as possible, the guest crews only stay for 30 minutes at one host, then they visit another host boat. The whole party lasts 3 hours.

Each crew has a given size of members and they always stay together, and each boat has a given capacity. The problem now is to assign all the guest crews to the host boats for each of the six time slots, using the minimum number of host boats.

The partitioning of crews into guests and hosts is fixed throughout the whole party and no two crews should meet more than once. For organizing reasons the boats 1 to 3 must be host boats, and the three boats 40 to 42 are guest boats. 

In [ ]:
def load_PPP(file_path):
    with open(file_path, 'r') as f:
        m, n = map(int, f.readline().split())
        
        capacities = list(map(int, f.readline().split()))
        
        crews = list(map(int, f.readline().split()))
        
        if len(capacities) != m or len(crews) != m:
            raise ValueError("Length not match!")
            
    return m, n, capacities, crews

args = load_PPP("../Assets/ProgressivePartyProblem/42_6.txt")
print(args)


(42, 6, [6, 8, 12, 12, 12, 12, 12, 10, 10, 10, 10, 10, 8, 8, 8, 12, 8, 8, 8, 8, 8, 8, 7, 7, 7, 7, 7, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 9, 0, 0, 0], [2, 2, 2, 2, 4, 4, 4, 1, 2, 2, 2, 3, 4, 2, 3, 6, 2, 2, 4, 2, 4, 5, 4, 4, 2, 2, 4, 5, 2, 4, 2, 2, 2, 2, 2, 2, 4, 5, 7, 2, 3, 4])


In [ ]:
from ortools.linear_solver import pywraplp
def main(args):
    m, n, capacities, crews = args

    print(m, n, capacities, crews)
    PPP = pywraplp.Solver.CreateSolver('SCIP')
    
    h = [PPP.BoolVar(f'h_{j}') for j in range(m)]
    # h_j, 1 if boat j is a host boat
    x = [[[PPP.BoolVar(f'x[{i}, {j}, {t}]') for t in range(n)] for j in range(m)] for i in range(m)]
    # x[i, j, t], 1 if guest boat j visits host boat i at time t
    
    u = [[[PPP.BoolVar(f'm[{i}, {j}, {t}]') for t in range(n)] for j in range(m)] for i in range(m)]
    # m[i, j, t], 1 if i and j are met at time t (Auxiliary variable)
    for i in range(m):
        for j in range(m):
            if i == j:
                continue
            for t in range(n):
                PPP.Add(x[i][j][t] <= h[i], name = f"x[{i}, {j}, {t}] <= h[{i}]")
                # Guest boat j can only visit host boat i if i is a host boat
    
    for i in range(m):
        for t in range(n):
            PPP.Add(sum([crews[j] * x[i][j][t] if i != j else 0 for j in range(m)]) <= capacities[i] * h[i], name = f"Capacity_{i}_{t}")
            # At time t, the total crew visiting host boat i cannot exceed its capacity
    
    for j in range(m):
        for t in range(n):
            PPP.Add(h[j] + sum(x[i][j][t] if i != j else 0 for i in range(m)) == 1, name = f"NoIdle_{j}_{t}")
    # Each boat is either a host boat or visits exactly one host boat at time t
    
    for i in range(m):
        for j in range(m):
            if i == j :
                continue
            else:
                PPP.Add(sum(x[i][j][t] for t in range(n)) <= h[i], name = f"VisitOnce_{i}_{j}")
    # Guest boat i can visit host boat j at most once
    
    for i in range(3):
        PPP.Add(h[i] == 1, name = f"HOST_{i}")
    
    for i in range(39, m):
        PPP.Add(h[i] == 0, name = f"HOST_{i}")
    # some boat are fixed as host boats or guest boats
    
    for i in range(m):
        for j in range(m):
            if i == j:
                continue
            for k in range(j + 1, m):
                if i == k:
                    continue 
                for t in range(n):
                    PPP.Add(u[j][k][t] >= x[i][j][t] + x[i][k][t] - 1, name = f"NoMoreOnce_{j}_{k}_{t}")
    
    for t in range(n):
        # for j in range(m):
        for j in range(m):
            PPP.Add(sum(u[j][k][t] for k in range(j + 1, m)) <= 1, name = f"NoMoreOnce2_{t}_{j}")
    # Guarantee that any two guest boats meet at most once
    
    PPP.Minimize(sum(h[j] for j in range(m)))
    
    status = PPP.Solve()
    if status == pywraplp.Solver.OPTIMAL:
        print('Objective value =', PPP.Objective().Value())
        print('Host boats:')
        for j in range(m):
            if h[j].solution_value() > 0.5:
                print(f'Boat {j} is a host boat.')
        print('Visits:')
        for i in range(m):
            for j in range(m):
                for t in range(n):
                    if x[i][j][t].solution_value() > 0.5:
                        print(f'Guest boat {i} visits host boat {j} at time {t}.')
    else:
        print('The problem does not have an optimal solution.')
main(args)

42 6 [6, 8, 12, 12, 12, 12, 12, 10, 10, 10, 10, 10, 8, 8, 8, 12, 8, 8, 8, 8, 8, 8, 7, 7, 7, 7, 7, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 9, 0, 0, 0] [2, 2, 2, 2, 4, 4, 4, 1, 2, 2, 2, 3, 4, 2, 3, 6, 2, 2, 4, 2, 4, 5, 4, 4, 2, 2, 4, 5, 2, 4, 2, 2, 2, 2, 2, 2, 4, 5, 7, 2, 3, 4]
